In [1]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from bs4 import BeautifulSoup as BS
from lxml import html
import re
import difflib
# from fuzzywuzzy import fuzz
# from fuzzywuzzy import process
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [2]:
imdbdf = pd.read_csv("../data/imdbdf.csv")

In [3]:
imdbdf = imdbdf.drop("Unnamed: 0",axis=1)

In [4]:
imdbdf.loc[(imdbdf['Title'] == 'SMILE')]

,Title,Release Year,Rating,Runtime,IMDB Score,Metascore
3,SMILE,2022.0,R,115.0,6.6,68
1006,SMILE,2009.0,Not Rated,84.0,3.3,NO METASCORE
1698,SMILE,2022.0,NO RATING,NaN,3.8,NO METASCORE


In [5]:
imdbdf = imdbdf.drop(1698)

In [6]:
imdbdf.loc[(imdbdf['Title'] == 'SMILE')]

,Title,Release Year,Rating,Runtime,IMDB Score,Metascore
3,SMILE,2022.0,R,115.0,6.6,68
1006,SMILE,2009.0,Not Rated,84.0,3.3,NO METASCORE


In [7]:
imdbdf['Release Year'] = pd.to_numeric(imdbdf['Release Year'])
imdbdf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9999 entries, 0 to 9999
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Title         9999 non-null   object 
 1   Release Year  9607 non-null   float64
 2   Rating        9999 non-null   object 
 3   Runtime       9234 non-null   float64
 4   IMDB Score    9999 non-null   object 
 5   Metascore     9999 non-null   object 
dtypes: float64(2), object(4)
memory usage: 546.8+ KB


In [8]:
imdbdf = imdbdf[imdbdf['Release Year'].notna()]

In [9]:
imdbdf.index = np.arange(1, len(imdbdf) + 1)
imdbdf['Title'] = imdbdf['Title'].str.upper()

In [10]:
imdbdf['Title'] = imdbdf['Title'].fillna(value=1408)

In [11]:
imdbdf.head(300)

,Title,Release Year,Rating,Runtime,IMDB Score,Metascore
1,THE MENU,2022.0,R,107.0,7.5,71
2,X,2022.0,R,105.0,6.6,79
3,BONES AND ALL,2022.0,R,131.0,7.3,74
4,SMILE,2022.0,R,115.0,6.6,68
5,BARBARIAN,2022.0,R,102.0,7.1,78
6,BHEDIYA,2022.0,NO RATING,156.0,7.7,NO METASCORE
7,NOPE,2022.0,R,130.0,6.9,77
8,THE MEAN ONE,2022.0,NO RATING,93.0,6.3,32
9,TERRIFIER 2,2022.0,Not Rated,138.0,6.3,59
10,PEARL,2022.0,R,103.0,7.0,73


In [12]:
wwbo = pd.read_excel("../data/thenumbers.xlsx", sheet_name=('All Time Worldwide Box Office'))
wwbo.index = np.arange(1, len(wwbo) + 1)
wwbo = wwbo.rename(columns={'Movie' : 'Title', 'Released' : 'Release Year'})
wwbo['Worldwide']=wwbo['Worldwide'].apply('{:,}'.format)
wwbo['Domestic']=wwbo['Domestic'].apply('{:,}'.format)
wwbo['International']=wwbo['International'].apply('{:,}'.format)

In [13]:
wwbo['Title'] = wwbo['Title'].str.upper()

In [14]:
wwbo

,Release Year,Title,Worldwide,Domestic,International
1,2017,IT,"701,012,746","328,828,874.0","372,183,872.0"
2,1975,JAWS,"482,947,378","272,257,035.0","210,690,343.0"
3,2007,I AM LEGEND,"585,532,684","256,393,010.0","329,139,674.0"
4,1973,THE EXORCIST,"428,214,478","230,347,346.0","197,867,132.0"
5,2019,IT: CHAPTER TWO,"467,563,955","211,593,228.0","255,970,727.0"
...,...,...,...,...,...
1851,2018,MANIAC TALES,155,nan,155.0
1852,1933,THE INVISIBLE MAN,132,nan,132.0
1853,2016,CAPTURE KILL RELEASE,72,nan,72.0
1854,2021,SLUMBER PARTY MASSACRE,28,nan,28.0


In [15]:
#df[['a', 'b']] = df[['a','b']].fillna(value=0)

wwbo['Title'] = wwbo['Title'].fillna(value=1408)

In [16]:
wwbo.head(100)

,Release Year,Title,Worldwide,Domestic,International
1,2017,IT,"701,012,746","328,828,874.0","372,183,872.0"
2,1975,JAWS,"482,947,378","272,257,035.0","210,690,343.0"
3,2007,I AM LEGEND,"585,532,684","256,393,010.0","329,139,674.0"
4,1973,THE EXORCIST,"428,214,478","230,347,346.0","197,867,132.0"
5,2019,IT: CHAPTER TWO,"467,563,955","211,593,228.0","255,970,727.0"
6,2018,A QUIET PLACE,"334,876,670","188,024,361.0","146,852,309.0"
7,2017,GET OUT,"252,297,405","175,837,935.0","76,459,470.0"
8,2001,HANNIBAL,"350,100,280","165,092,266.0","185,008,014.0"
9,2021,A QUIET PLACE: PART II,"296,650,356","160,215,764.0","136,434,592.0"
10,2018,HALLOWEEN,"255,416,089","159,366,015.0","96,050,074.0"


In [17]:
tdbo = pd.read_excel("../data/thenumbers.xlsx", sheet_name=('Top 100 Domestic Box Office'))
tdbo.index = np.arange(1, len(tdbo) + 1)
tdbo = tdbo.rename(columns={'Movie' : 'Title', 'Released' : 'Release Year', 'Infl. Adj. Dom.' : 'Domestic - Adjusted for Inflation'})
tdbo['Domestic']=tdbo['Domestic'].apply('{:,}'.format)
tdbo['Domestic - Adjusted for Inflation']=tdbo['Domestic - Adjusted for Inflation'].apply('{:,}'.format)
tdbo['Title'] = tdbo['Title'].str.upper()

In [18]:
tdbo['Title'] = tdbo['Title'].fillna(value=1408)

In [19]:
tdbo

,Release Year,Title,Domestic,Domestic - Adjusted for Inflation
1,2017,IT,"328,828,874","334,783,459"
2,1973,THE EXORCIST,"230,347,346","1,063,304,892"
3,2019,IT CHAPTER TWO,"211,593,228","211,824,221"
4,2018,A QUIET PLACE,"188,024,361","189,262,720"
5,2017,GET OUT,"175,837,935","179,752,944"
6,2020,A QUIET PLACE PART II,"160,215,764","160,215,764"
7,2018,HALLOWEEN,"159,366,015","160,409,948"
8,1999,THE BLAIR WITCH PROJECT,"140,539,099","253,689,664"
9,2016,SPLIT,"138,141,585","141,221,659"
10,2013,THE CONJURING,"137,400,141","154,976,540"


In [20]:
top100comb = pd.merge(tdbo, imdbdf, left_on = ['Title', 'Release Year'], right_on = ['Title', 'Release Year'], how='left')
top100comb.index = np.arange(1, len(top100comb) + 1)

In [21]:
top100comb

,Release Year,Title,Domestic,Domestic - Adjusted for Inflation,Rating,Runtime,IMDB Score,Metascore
1,2017,IT,"328,828,874","334,783,459",R,135.0,7.3,69
2,1973,THE EXORCIST,"230,347,346","1,063,304,892",R,122.0,8.1,81
3,2019,IT CHAPTER TWO,"211,593,228","211,824,221",R,169.0,6.5,58
4,2018,A QUIET PLACE,"188,024,361","189,262,720",PG-13,90.0,7.5,82
5,2017,GET OUT,"175,837,935","179,752,944",R,104.0,7.7,85
6,2020,A QUIET PLACE PART II,"160,215,764","160,215,764",PG-13,97.0,7.2,71
7,2018,HALLOWEEN,"159,366,015","160,409,948",R,106.0,6.5,67
8,1999,THE BLAIR WITCH PROJECT,"140,539,099","253,689,664",R,81.0,6.5,81
9,2016,SPLIT,"138,141,585","141,221,659",PG-13,117.0,7.3,62
10,2013,THE CONJURING,"137,400,141","154,976,540",R,112.0,7.5,68


In [22]:
# worldwidecomb = pd.merge(imdbdf, wwbo, left_on = ['Title', 'Release Year'], right_on = ['Title', 'Release Year'], how='inner')
# worldwidecomb.index = np.arange(1, len(top100comb) + 1)

In [23]:
imdbdf['Release Year'] = imdbdf['Release Year'].astype(str)

In [24]:
imdbdf.dtypes

Title            object
Release Year     object
Rating           object
Runtime         float64
IMDB Score       object
Metascore        object
dtype: object

In [25]:
imdbdf['Title and Year'] = imdbdf['Title'] + " " + imdbdf['Release Year']

In [26]:
imdbdf.head()

,Title,Release Year,Rating,Runtime,IMDB Score,Metascore,Title and Year
1,THE MENU,2022.0,R,107.0,7.5,71,THE MENU 2022.0
2,X,2022.0,R,105.0,6.6,79,X 2022.0
3,BONES AND ALL,2022.0,R,131.0,7.3,74,BONES AND ALL 2022.0
4,SMILE,2022.0,R,115.0,6.6,68,SMILE 2022.0
5,BARBARIAN,2022.0,R,102.0,7.1,78,BARBARIAN 2022.0


In [27]:
wwbo['Release Year'] = wwbo['Release Year'].astype(str)

In [28]:
wwbo['Title'] = wwbo['Title'].astype(str)

In [29]:
wwbo['Title and Year'] = wwbo['Title'] + " " + wwbo['Release Year']

In [30]:
wwbo

,Release Year,Title,Worldwide,Domestic,International,Title and Year
1,2017,IT,"701,012,746","328,828,874.0","372,183,872.0",IT 2017
2,1975,JAWS,"482,947,378","272,257,035.0","210,690,343.0",JAWS 1975
3,2007,I AM LEGEND,"585,532,684","256,393,010.0","329,139,674.0",I AM LEGEND 2007
4,1973,THE EXORCIST,"428,214,478","230,347,346.0","197,867,132.0",THE EXORCIST 1973
5,2019,IT: CHAPTER TWO,"467,563,955","211,593,228.0","255,970,727.0",IT: CHAPTER TWO 2019
...,...,...,...,...,...,...
1851,2018,MANIAC TALES,155,nan,155.0,MANIAC TALES 2018
1852,1933,THE INVISIBLE MAN,132,nan,132.0,THE INVISIBLE MAN 1933
1853,2016,CAPTURE KILL RELEASE,72,nan,72.0,CAPTURE KILL RELEASE 2016
1854,2021,SLUMBER PARTY MASSACRE,28,nan,28.0,SLUMBER PARTY MASSACRE 2021


In [31]:
wwbo.dtypes

Release Year      object
Title             object
Worldwide         object
Domestic          object
International     object
Title and Year    object
dtype: object

In [32]:
wwbo

,Release Year,Title,Worldwide,Domestic,International,Title and Year
1,2017,IT,"701,012,746","328,828,874.0","372,183,872.0",IT 2017
2,1975,JAWS,"482,947,378","272,257,035.0","210,690,343.0",JAWS 1975
3,2007,I AM LEGEND,"585,532,684","256,393,010.0","329,139,674.0",I AM LEGEND 2007
4,1973,THE EXORCIST,"428,214,478","230,347,346.0","197,867,132.0",THE EXORCIST 1973
5,2019,IT: CHAPTER TWO,"467,563,955","211,593,228.0","255,970,727.0",IT: CHAPTER TWO 2019
...,...,...,...,...,...,...
1851,2018,MANIAC TALES,155,nan,155.0,MANIAC TALES 2018
1852,1933,THE INVISIBLE MAN,132,nan,132.0,THE INVISIBLE MAN 1933
1853,2016,CAPTURE KILL RELEASE,72,nan,72.0,CAPTURE KILL RELEASE 2016
1854,2021,SLUMBER PARTY MASSACRE,28,nan,28.0,SLUMBER PARTY MASSACRE 2021


In [33]:
wwbo.to_csv('wwbo.csv')

In [34]:
imdbdf.dtypes

Title              object
Release Year       object
Rating             object
Runtime           float64
IMDB Score         object
Metascore          object
Title and Year     object
dtype: object

In [35]:
imdbdf['Runtime'] = imdbdf['Runtime'].astype(str)

In [36]:
imdbdf

,Title,Release Year,Rating,Runtime,IMDB Score,Metascore,Title and Year
1,THE MENU,2022.0,R,107.0,7.5,71,THE MENU 2022.0
2,X,2022.0,R,105.0,6.6,79,X 2022.0
3,BONES AND ALL,2022.0,R,131.0,7.3,74,BONES AND ALL 2022.0
4,SMILE,2022.0,R,115.0,6.6,68,SMILE 2022.0
5,BARBARIAN,2022.0,R,102.0,7.1,78,BARBARIAN 2022.0
...,...,...,...,...,...,...,...
9603,WALKING DEAD - TOMATE,2022.0,NO RATING,79.0,3.6,NO METASCORE,WALKING DEAD - TOMATE 2022.0
9604,MANNEQUIN,2023.0,NO RATING,nan,NO IMDB RATING,NO METASCORE,MANNEQUIN 2023.0
9605,UNDEAD OR ALIVE: A ZOMBEDY,2007.0,R,91.0,5.1,NO METASCORE,UNDEAD OR ALIVE: A ZOMBEDY 2007.0
9606,SECOND ORIGIN,2015.0,NO RATING,106.0,4.5,NO METASCORE,SECOND ORIGIN 2015.0


In [37]:
# worldwidecomb = pd.merge(imdbdf, wwbo, left_on = ['Title', 'Release Year'], right_on = ['Title', 'Release Year'], how='inner')
# worldwidecomb.index = np.arange(1, len(top100comb) + 1)

# df2['team'] = df2['team'].apply(lambda x: difflib.get_close_matches(x, df1['team'])[0])

#wwbo['Title and Year'] = wwbo['Title and Year'].map(lambda x: difflib.get_close_matches(x, imdbdf['Title and Year'])[0])

#test = imdbdf.merge(wwbo)

#df2.index = df2.index.map(lambda x: difflib.get_close_matches(x, df1.index)[0])

# for x in wwbo['Title and Year']:
#     difflib.get_close_matches(x, imdbdf['Title and Year'])[0]
#     print(x)
#     print(difflib.get_close_matches(x, imdbdf['Title and Year'])[0])

IT 2017
IT 2017.0
JAWS 1975
JAWS 2 1978.0
I AM LEGEND 2007
BIG LEGEND 2018.0
THE EXORCIST 1973
THE EXORCIST 1973.0
IT: CHAPTER TWO 2019
IT CHAPTER TWO 2019.0
A QUIET PLACE 2018
A QUIET PLACE 2018.0
GET OUT 2017
GET OUT 2017.0
HANNIBAL 2001
CANNIBAL 2010.0
A QUIET PLACE: PART II 2021
A QUIET PLACE PART II 2020.0
HALLOWEEN 2018
HALLOWEEN 2018.0
THE BLAIR WITCH PROJECT 1999
THE BLAIR WITCH PROJECT 1999.0
SPLIT 2017
SPLIT 2016.0
THE CONJURING 2013
THE CONJURING 2013.0
THE SILENCE OF THE LAMBS 1991
THE SHIVER OF THE VAMPIRES 1971.0
THE RING 2002
THE RING 2002.0
NOPE 2022
NOPE 2022.0
THE NUN 2018
THE NUN 2018.0
THE GRUDGE 2004
THE GRUDGE 2004.0
PARANORMAL ACTIVITY 2009
PARANORMAL ACTIVITY 2007.0
SMILE 2022
SMILE 2022.0
INTERVIEW WITH THE VAMPIRE 1994
INTERVIEW WITH THE VAMPIRE: THE VAMPIRE CHRONICLES 1994.0
PARANORMAL ACTIVITY 3 2011
PARANORMAL ACTIVITY 3 2011.0
SCREAM 1996
SCREAM 1996.0
JAWS 2 1978
JAWS 2 1978.0
THE CONJURING 2: THE ENFIELD POLTERGEIST 2016
THE CONJURING: THE DEVIL MADE ME 

AN AMERICAN WEREWOLF IN LONDON 1981.0
LAKE PLACID 1999
LAKE PLACID 1999.0
QUARANTINE 2008
QUARANTINE 2008.0
HOUSE AT THE END OF THE STREET 2012
HOUSE AT THE END OF THE STREET 2012.0
DOCTOR SLEEP 2019
DOCTOR SLEEP 2019.0
ANACONDAS: THE HUNT FOR THE BLOOD ORCHID 2004
ANACONDAS: THE HUNT FOR THE BLOOD ORCHID 2004.0
THE EYE 2008
THE EYE 2008.0
CRIMSON PEAK 2015
CRIMSON PEAK 2015.0
MIRRORS 2008
MIRRORS 2008.0
DELIVER US FROM EVIL 2014
DELIVER US FROM EVIL 2014.0
SLENDER MAN 2018
SLENDER MAN 2018.0
SLEEPWALKERS 1992
SLEEPWALKERS 1992.0
QUEEN OF THE DAMNED 2002
QUEEN OF THE DAMNED 2002.0
HALLOWEEN: RESURRECTION 2002
HALLOWEEN: RESURRECTION 2002.0
GHOST SHIP 2002
GHOST SHIP 2002.0
DAYBREAKERS 2010
DAYBREAKERS 2009.0
THE FOG 2005
THE FOG 2005.0
TWILIGHT ZONE: THE MOVIE 1983
TWILIGHT ZONE: THE MOVIE 1983.0
BLESS THE CHILD 2000
BLESS THE CHILD 2000.0
CHILD’S PLAY 2019
CHILD'S PLAY 2019.0
PRIEST 2011
PRIEST 2011.0
28 WEEKS LATER 2007
28 WEEKS LATER 2007.0
THE UNINVITED 2009
THE UNINVITED 2009.0
DO

THE RETURN OF THE LIVING DEAD 1985
THE RETURN OF THE LIVING DEAD 1985.0
BUFFY THE VAMPIRE SLAYER 1992
BUFFY THE VAMPIRE SLAYER 1992.0
SCANNERS 1981
SCANNERS 1981.0
PRINCE OF DARKNESS 1987
PRINCE OF DARKNESS 1987.0
POLTERGEIST III 1988
POLTERGEIST III 1988.0
FLOWERS IN THE ATTIC 1987
SPIDER IN THE ATTIC 2021.0
CREEPSHOW 2 1987
CREEPSHOW 2 1987.0
IT COMES AT NIGHT 2017
IT COMES AT NIGHT 2017.0
CANDYMAN: FAREWELL TO FLESH 1995
CANDYMAN: FAREWELL TO THE FLESH 1995.0
THE THING 1982
THE THING 1982.0
THE HAUNTING OF MOLLY HARTLEY 2008
THE HAUNTING OF MOLLY HARTLEY 2008.0
MALIGNANT 2021
MALIGNANT 2021.0
THE ENTITY 1983
THE ENTITY 1982.0
CASE 39 2010
CASE 39 2009.0
LORD OF ILLUSIONS 1995
LORD OF ILLUSIONS 1995.0
FEARDOTCOM 2002
FEARDOTCOM 2002.0
CRITTERS 1986
CRITTERS 1986.0
JASON X 2002
JASON X 2001.0
THE INCUBUS 1982
THE INCUBUS 1981.0
THE GATE 1987
THE GATE 1987.0
MAN'S BEST FRIEND 1993
MAN'S BEST FRIEND 1993.0
APRIL FOOL'S DAY 1986
APRIL FOOL'S DAY 1986.0
THEY 2002
THEY 2002.0
SILENT HOUSE 

IndexError: list index out of range

In [38]:
imdbdf.loc[(imdbdf['Title'] == '1408')]

,Title,Release Year,Rating,Runtime,IMDB Score,Metascore,Title and Year
325,1408,2007.0,PG-13,104.0,6.8,64,1408 2007.0


In [39]:
wwbo.loc[(wwbo['Title'] == '1408')]

,Release Year,Title,Worldwide,Domestic,International,Title and Year
55,2007,1408,"131,263,370","71,985,628.0","59,277,742.0",1408 2007
594,2016,1408,"922,727","779,820.0","142,907.0",1408 2016
744,2011,1408,"2,756,875","38,100.0","2,718,775.0",1408 2011
959,2018,1408,"3,130,260",nan,"3,130,260.0",1408 2018
1347,2020,1408,"123,784",nan,"123,784.0",1408 2020


In [40]:
#imdbdf = imdbdf.drop(1698) drop extra 1408

In [41]:
wwbo = wwbo.drop(594)
wwbo = wwbo.drop(744)
wwbo = wwbo.drop(959)
wwbo = wwbo.drop(1347)

In [42]:
wwbo.loc[(wwbo['Title'] == '1408')]

,Release Year,Title,Worldwide,Domestic,International,Title and Year
55,2007,1408,"131,263,370","71,985,628.0","59,277,742.0",1408 2007


In [ ]:
##1408 is returning too many times in wwbo because of NAN replacement. convert the others to string instead of floats and try fuzzy matching again. 

In [43]:
wwbo.head()

,Release Year,Title,Worldwide,Domestic,International,Title and Year
1,2017,IT,"701,012,746","328,828,874.0","372,183,872.0",IT 2017
2,1975,JAWS,"482,947,378","272,257,035.0","210,690,343.0",JAWS 1975
3,2007,I AM LEGEND,"585,532,684","256,393,010.0","329,139,674.0",I AM LEGEND 2007
4,1973,THE EXORCIST,"428,214,478","230,347,346.0","197,867,132.0",THE EXORCIST 1973
5,2019,IT: CHAPTER TWO,"467,563,955","211,593,228.0","255,970,727.0",IT: CHAPTER TWO 2019


In [44]:
wwbo.shape

(1851, 6)

In [45]:
imdbdf.head()

,Title,Release Year,Rating,Runtime,IMDB Score,Metascore,Title and Year
1,THE MENU,2022.0,R,107.0,7.5,71,THE MENU 2022.0
2,X,2022.0,R,105.0,6.6,79,X 2022.0
3,BONES AND ALL,2022.0,R,131.0,7.3,74,BONES AND ALL 2022.0
4,SMILE,2022.0,R,115.0,6.6,68,SMILE 2022.0
5,BARBARIAN,2022.0,R,102.0,7.1,78,BARBARIAN 2022.0


In [46]:
imdbdf.shape

(9607, 7)

In [ ]:
# title a / title b / calc ratio of similarity - abi's suggestion

#df['candidate'] = get_close_matches()

In [ ]:
# worldwidecomb = pd.merge(imdbdf, wwbo, left_on = ['Title', 'Release Year'], right_on = ['Title', 'Release Year'], how='inner')
# worldwidecomb.index = np.arange(1, len(top100comb) + 1)

# df2['team'] = df2['team'].apply(lambda x: difflib.get_close_matches(x, df1['team'])[0])

#wwbo['Title and Year'] = wwbo['Title and Year'].map(lambda x: difflib.get_close_matches(x, imdbdf['Title and Year'])[0])

#test = imdbdf.merge(wwbo)

#df2.index = df2.index.map(lambda x: difflib.get_close_matches(x, df1.index)[0])

# for x in wwbo['Title and Year']:
#     wwbo['Match'] = difflib.get_close_matches(x, imdbdf['Title and Year'])[0]
#     print(x)
#     print(difflib.get_close_matches(x, imdbdf['Title and Year'])[0])

In [47]:
wwbo

,Release Year,Title,Worldwide,Domestic,International,Title and Year
1,2017,IT,"701,012,746","328,828,874.0","372,183,872.0",IT 2017
2,1975,JAWS,"482,947,378","272,257,035.0","210,690,343.0",JAWS 1975
3,2007,I AM LEGEND,"585,532,684","256,393,010.0","329,139,674.0",I AM LEGEND 2007
4,1973,THE EXORCIST,"428,214,478","230,347,346.0","197,867,132.0",THE EXORCIST 1973
5,2019,IT: CHAPTER TWO,"467,563,955","211,593,228.0","255,970,727.0",IT: CHAPTER TWO 2019
...,...,...,...,...,...,...
1851,2018,MANIAC TALES,155,nan,155.0,MANIAC TALES 2018
1852,1933,THE INVISIBLE MAN,132,nan,132.0,THE INVISIBLE MAN 1933
1853,2016,CAPTURE KILL RELEASE,72,nan,72.0,CAPTURE KILL RELEASE 2016
1854,2021,SLUMBER PARTY MASSACRE,28,nan,28.0,SLUMBER PARTY MASSACRE 2021


In [ ]:
# df['bk'] = df['title'].str[0:2]

# .replace(';', '')

In [48]:
#df.to_csv('file1.csv')

In [49]:
wwbo.to_csv('wwbo.csv')

In [50]:
imdbdf.to_csv('imdbdf.csv')

In [51]:
wwbo['match'] = wwbo['Title'].str[0:2]

In [52]:
wwbo

,Release Year,Title,Worldwide,Domestic,International,Title and Year,match
1,2017,IT,"701,012,746","328,828,874.0","372,183,872.0",IT 2017,IT
2,1975,JAWS,"482,947,378","272,257,035.0","210,690,343.0",JAWS 1975,JA
3,2007,I AM LEGEND,"585,532,684","256,393,010.0","329,139,674.0",I AM LEGEND 2007,I
4,1973,THE EXORCIST,"428,214,478","230,347,346.0","197,867,132.0",THE EXORCIST 1973,TH
5,2019,IT: CHAPTER TWO,"467,563,955","211,593,228.0","255,970,727.0",IT: CHAPTER TWO 2019,IT
...,...,...,...,...,...,...,...
1851,2018,MANIAC TALES,155,nan,155.0,MANIAC TALES 2018,MA
1852,1933,THE INVISIBLE MAN,132,nan,132.0,THE INVISIBLE MAN 1933,TH
1853,2016,CAPTURE KILL RELEASE,72,nan,72.0,CAPTURE KILL RELEASE 2016,CA
1854,2021,SLUMBER PARTY MASSACRE,28,nan,28.0,SLUMBER PARTY MASSACRE 2021,SL


In [57]:
imdbdf['match'] = imdbdf['Title and Year'].str[0:4]

In [61]:
imdbdf.head()

,Title,Release Year,Rating,Runtime,IMDB Score,Metascore,Title and Year,match
1,THE MENU,2022.0,R,107.0,7.5,71,THE MENU 2022.0,THE
2,X,2022.0,R,105.0,6.6,79,X 2022.0,X 20
3,BONES AND ALL,2022.0,R,131.0,7.3,74,BONES AND ALL 2022.0,BONE
4,SMILE,2022.0,R,115.0,6.6,68,SMILE 2022.0,SMIL
5,BARBARIAN,2022.0,R,102.0,7.1,78,BARBARIAN 2022.0,BARB


In [59]:
wwbo['match'] = wwbo['Title and Year'].str[0:4]

In [60]:
wwbo.head()

,Release Year,Title,Worldwide,Domestic,International,Title and Year,match
1,2017,IT,"701,012,746","328,828,874.0","372,183,872.0",IT 2017,IT 2
2,1975,JAWS,"482,947,378","272,257,035.0","210,690,343.0",JAWS 1975,JAWS
3,2007,I AM LEGEND,"585,532,684","256,393,010.0","329,139,674.0",I AM LEGEND 2007,I AM
4,1973,THE EXORCIST,"428,214,478","230,347,346.0","197,867,132.0",THE EXORCIST 1973,THE
5,2019,IT: CHAPTER TWO,"467,563,955","211,593,228.0","255,970,727.0",IT: CHAPTER TWO 2019,IT:
